<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [4]:
!pip install gensim

In [6]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 67.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arviz 0.22.0 requires matplotlib>=3.8, which is not installed.
bigframes 2.19.0 requires matplotlib>=3.7.1, which is not installed.
geemap 0.35.3 requires matplotlib, which is not installed.
prophet 1.1.7 requires matplotlib>=2.0.0, which is not installed.
mlxtend 0.23.4 requires matplotlib>=3.0.0, which is not installed.
fastai 2.8.4 requires matplotlib, which is not installed.
plotnine 0.14.5 requires matplotlib>=3.8.0, which is not installed.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
dask-cudf-cu12 25.6.0 req

In [7]:
!pip install matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.1 MB/s eta 0:00:00


In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [9]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

--2025-09-18 12:22:31--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2025-09-18 12:22:31--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2025-09-18 12:22:32--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [10]:
# Posibles bandas
os.listdir("./songs_dataset/")

['patti-smith.txt',
 'nickelback.txt',
 'missy-elliott.txt',
 'bjork.txt',
 'bob-dylan.txt',
 'beatles.txt',
 'lorde.txt',
 'blink-182.txt',
 'paul-simon.txt',
 'jimi-hendrix.txt',
 'amy-winehouse.txt',
 'prince.txt',
 'dickinson.txt',
 'eminem.txt',
 'leonard-cohen.txt',
 'radiohead.txt',
 'britney-spears.txt',
 'dj-khaled.txt',
 'adele.txt',
 'drake.txt',
 'lil-wayne.txt',
 'kanye-west.txt',
 'bruno-mars.txt',
 'janisjoplin.txt',
 'lin-manuel-miranda.txt',
 'bieber.txt',
 'ludacris.txt',
 'joni-mitchell.txt',
 'nicki-minaj.txt',
 'dr-seuss.txt',
 'notorious-big.txt',
 'r-kelly.txt',
 'bob-marley.txt',
 'kanye.txt',
 'nirvana.txt',
 'dolly-parton.txt',
 'rihanna.txt',
 'disney.txt',
 'nursery_rhymes.txt',
 'bruce-springsteen.txt',
 'michael-jackson.txt',
 'cake.txt',
 'notorious_big.txt',
 'al-green.txt',
 'Lil_Wayne.txt',
 'lady-gaga.txt',
 'Kanye_West.txt',
 'alicia-keys.txt',
 'johnny-cash.txt']

In [ ]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)
df.head()

In [ ]:
print("Cantidad de documentos:", df.shape[0])

### 1 - Preprocesamiento

In [ ]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:2]

### 2 - Crear los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

### 3 - Entrenar embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

In [11]:
 #Armar el dataset utilizando salto de línea para separar las oraciones/docs
df_adele = pd.read_csv('songs_dataset/adele.txt', sep='/n', header=None)
df_adele.head()

/tmp/ipython-input-350300306.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_adele = pd.read_csv('songs_dataset/adele.txt', sep='/n', header=None)


,0
0,Looking for some education
1,Made my way into the night
2,All that bullshit conversation
3,"Baby, can't you read the signs? I won't bore y..."
4,I don't even wanna waste your time


In [13]:
print("Cantidad de documentos_adele:", df_adele.shape[0])

Cantidad de documentos_adele: 2400


### 1 - Preprocesamiento

In [15]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens_adele = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df_adele[:None].iterrows():
    sentence_tokens_adele.append(text_to_word_sequence(row[0]))

In [16]:
# Demos un vistazo
sentence_tokens_adele[:2]

[['looking', 'for', 'some', 'education'],
 ['made', 'my', 'way', 'into', 'the', 'night']]

### 2 - Crear los vectores (word2vec)

In [17]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [19]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model_adele = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [20]:
# Obtener el vocabulario con los tokens
w2v_model_adele.build_vocab(sentence_tokens_adele)


In [21]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model_adele.corpus_count)

Cantidad de docs en el corpus: 2400


In [22]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model_adele.wv.index_to_key))

Cantidad de words distintas en el corpus: 516


# Entrenamiento

In [23]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model_adele.train(
    sentence_tokens_adele,
    total_examples=w2v_model_adele.corpus_count,
    epochs=20,
    compute_loss=True,
    callbacks=[callback()]
)


Loss after epoch 0: 156064.734375
Loss after epoch 1: 94113.765625
Loss after epoch 2: 92955.625
Loss after epoch 3: 92854.875
Loss after epoch 4: 92871.3125
Loss after epoch 5: 94204.5625
Loss after epoch 6: 91984.9375
Loss after epoch 7: 88876.8125
Loss after epoch 8: 87212.4375
Loss after epoch 9: 85029.4375
Loss after epoch 10: 81237.875
Loss after epoch 11: 72765.125
Loss after epoch 12: 71033.625
Loss after epoch 13: 70587.875
Loss after epoch 14: 70159.875
Loss after epoch 15: 68915.875
Loss after epoch 16: 68236.0
Loss after epoch 17: 68022.25
Loss after epoch 18: 68458.5
Loss after epoch 19: 67770.375


(218266, 380480)

# Ensayar

In [28]:
# Palabras que MÁS se relacionan con...:
w2v_model_adele.wv.most_similar(positive=["break"], topn=10)

[('hearts', 0.981159508228302),
 ('wait', 0.962829053401947),
 ("they're", 0.960909903049469),
 ('watch', 0.9574174284934998),
 ('piece', 0.9550704956054688),
 ('part', 0.9529026746749878),
 ('old', 0.9502490758895874),
 ('getting', 0.947179913520813),
 ('finally', 0.9466720819473267),
 ('moment', 0.9466462731361389)]

Estas palabras similares expresan:

1-La idea de romper corazones (break hearts,) Y un sentimiento de espera pasivo en el tiempo (wait, moment)

2-Por otra parte semánticamente lo relacionó con piece y part

In [29]:
# Palabras que MENOS se relacionan con...:
w2v_model_adele.wv.most_similar(negative=["break"], topn=10)

[('ready', -0.44789254665374756),
 ('her', -0.46378761529922485),
 ('wish', -0.48194748163223267),
 ('gonna', -0.4836290776729584),
 ("you're", -0.4932636618614197),
 ('when', -0.4952651858329773),
 ('treat', -0.49613940715789795),
 ('better', -0.50475013256073),
 ('one', -0.5062729120254517),
 ('miss', -0.5129149556159973)]

Acá no lo relaciona con esperanza o deseos (wish, miss ) ni con  un futuro activo ya que estos verbos que están menos relacionados expresan acción ( going to, treat)  y siguiendo con la idea de espera pasiva  marcada por la palabras menos relacionada "ready "

In [31]:
# Palabras que MÁS se relacionan con...:
w2v_model_adele.wv.most_similar(positive=["always"], topn=10)

[('win', 0.9216226935386658),
 ('would', 0.8944078683853149),
 ('promise', 0.7749091386795044),
 ('stay', 0.7524643540382385),
 ('however', 0.7514536380767822),
 ('remedy', 0.749398946762085),
 ("i'd", 0.746101438999176),
 ('he', 0.7387968897819519),
 ('still', 0.7299801111221313),
 ('long', 0.7176405787467957)]

Relación emocional y morfosintáctica= deseos, promesas, algo que hubiera pasado (would, promise)

Otra vez la idea de pasividad en la espera (stay)


In [32]:
# Palabras que MENOS se relacionan con...:
w2v_model_adele.wv.most_similar(negative=["always"], topn=10)

[('the', -0.1630517840385437),
 ('sky', -0.18635150790214539),
 ('take', -0.18793517351150513),
 ('night', -0.1942976713180542),
 ('together', -0.2027338147163391),
 ('skyfall', -0.2041073441505432),
 ('lights', -0.21348078548908234),
 ('fall', -0.22194665670394897),
 ('gotta', -0.22717465460300446),
 ('rolling', -0.23252888023853302)]

No se relaciona con acciones que el sujeto realiza activamente (always take)  ni con espacios luminosos o" buenos"  como sky, lights.   

La palabra together podría expresar  la idea de soledad .


In [33]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model_adele.wv.most_similar(negative=["happy"])

KeyError: "Key 'happy' not present in vocabulary"

Happy  no es un término que se use en el tipo de letra de Adele ya que sus sentimientos son de  dolor intenso o de amores extensos terminados o de  angustia tras las ropturas amorosas.

In [35]:
# el método `get_vector` permite obtener los vectores:
vector_break_adele = w2v_model_adele.wv.get_vector("break")
print(vector_break_adele)

[-0.02380449  0.0996833  -0.10738026  0.08760989 -0.11226208  0.03955043
  0.1788185   0.21351463 -0.01002653  0.0144475   0.09448683 -0.01138666
  0.0645882  -0.01714084  0.02500312 -0.17147641  0.18574306 -0.13031483
  0.0686506   0.01426189 -0.01728987 -0.03944006  0.04131838  0.14216231
  0.15365876  0.02658117 -0.09731677 -0.06513214 -0.13812232  0.04856484
  0.11163878 -0.00206231  0.05877159 -0.07481293  0.01454056  0.07197302
  0.08594699 -0.1542946  -0.12938513  0.10291304 -0.00465012 -0.11009993
 -0.01506857  0.07155253  0.18025158  0.10753044 -0.0459106   0.02803144
  0.04257443  0.13233621 -0.05164624  0.08323949 -0.02581533  0.0436778
 -0.04901412  0.0142749  -0.03183051 -0.06524665 -0.0587381   0.07643826
  0.00363653 -0.14546083  0.03437543 -0.02755503  0.11491017  0.13188766
  0.09942731  0.15804686 -0.1209056  -0.08904403  0.18272299 -0.01245281
  0.04202103  0.04331023  0.06075816 -0.02633542 -0.14812563 -0.00556749
 -0.02801869  0.08700354 -0.18154764 -0.12442389  0.

In [37]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model_adele.wv.most_similar(vector_break_adele)

[('break', 1.0000001192092896),
 ('hearts', 0.981159508228302),
 ('wait', 0.9628289937973022),
 ("they're", 0.9609097838401794),
 ('watch', 0.9574174284934998),
 ('piece', 0.9550704956054688),
 ('part', 0.952902615070343),
 ('old', 0.9502490758895874),
 ('getting', 0.947179913520813),
 ('finally', 0.9466720819473267)]

Los vectores  similares expresan el sentimiento de las canciones de Adele de ruptura de corazones (break,piece,part, get break heart) y  espera pasiva  (wait, watch)  al igual que  la búsqueda por las palabras que más se relacionan del codigo anterior.

In [48]:
# el método `get_vector` permite obtener los vectores:
vector_always_adele = w2v_model_adele.wv.get_vector("always")
print(vector_always_adele)

[-0.0908862   0.20485173 -0.15531148 -0.26342887 -0.09984998 -0.0619072
  0.25671092  0.08579756  0.42748967 -0.02176334 -0.10652056  0.02356089
 -0.14784743  0.03998227 -0.11275868 -0.0052334   0.20907563 -0.08565539
 -0.0971342  -0.01718638 -0.11153125 -0.19697899 -0.06397893  0.34336516
 -0.04793438  0.26274905 -0.25561112  0.07410721 -0.25736365  0.20327544
  0.12768304  0.12253608 -0.17805634 -0.07718696 -0.15271145  0.09067912
 -0.03770993 -0.23785108 -0.4873347   0.21478683  0.19212301 -0.20047136
  0.15451987 -0.13911198  0.4666366   0.15434249  0.04071083 -0.00403251
  0.11162946  0.22016335  0.02534015 -0.1074343   0.21423785  0.0896607
 -0.09627452 -0.0539974   0.2296376  -0.22065195 -0.11656207  0.01124181
  0.24781984 -0.3092614   0.20802397 -0.15381414 -0.17008403  0.36154675
  0.14716841 -0.06206972 -0.29532108 -0.11014932  0.15127356  0.0334827
 -0.00451769 -0.15761352  0.26686832  0.0582644  -0.14843573  0.08976056
 -0.24581979 -0.03198067 -0.5359654  -0.08201324 -0.11

In [49]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model_adele.wv.most_similar(vector_always_adele)

[('always', 0.9999999403953552),
 ('win', 0.9216226935386658),
 ('would', 0.8944078683853149),
 ('promise', 0.7749091386795044),
 ('stay', 0.7524642944335938),
 ('however', 0.7514536380767822),
 ('remedy', 0.749398946762085),
 ("i'd", 0.746101438999176),
 ('he', 0.7387968897819519),
 ('still', 0.7299801111221313)]

Esta similitud de vectores nuevamente expresan los sentimientos de las canciones de Adele (espera , algo que hubiera pasado , promesas)

**El método most_similar puede usarse tanto con palabras como con vectores. En este caso, al pasar el vector de "break" y "always" directamente, se obtienen los mismos resultados que al usar la palabra, lo que confirma que el modelo está trabajando con la misma representación interna**

# Visualización de vectores

In [52]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np
def reduce_dimensions_adele(model, num_dimensions=2):
    vectors_adele = np.asarray(model.wv.vectors)
    labels_adele = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors_adele = tsne.fit_transform(vectors_adele)

    return vectors_adele, labels_adele

In [53]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px
vecs, labels = reduce_dimensions_adele(w2v_model_adele)
MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

# Se observa patrones reales:

Zona del dolor: broken, heart, tears, cry, hurt, miss, alone

Zona de la separación: leave, go, away, gone, without, never

Zona del tiempo: time, when, then, before, now, always

Zona de la memoria: remember, know, think, feel, was, had

El modelo aprendió temas emocionales claves de Adele.

In [54]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions_adele(w2v_model_adele,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

# Esta gráfica no solo muestra palabras… muestra emociones.
'Break' está en el centro del dolor, rodeado por lágrimas, ausencias y silencios.
'Always' está en la cima de la espera, rodeado por promesas que no se cumplieron.
El modelo, con precisión matemática, nos revela que en Adele, el amor no termina con un adiós  sino con un always .

**Para finalizar, es importante destacar que, aunque el modelo Word2Vec logra identificar asociaciones semánticas con alta precisión, estas adquieren verdadero significado solo cuando son interpretadas por el ojo humano. Es la experiencia, el contexto cultural y la sensibilidad emocional del analista lo que permite comprender  el universo de Adele**

In [55]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors_adele = np.asarray(w2v_model_adele.wv.vectors)
labels_adele = list(w2v_model_adele.wv.index_to_key)

np.savetxt("vectors.tsv", vectors_adele, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

In [56]:
from google.colab import files

files.download("vectors.tsv")
files.download("labels.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>